In [ ]:
import requests
%pylab inline
import pandas as pd
import json
matplotlib.rcParams['font.family'] = ['STHeiti']
matplotlib.rcParams['font.size'] = 16.0

In [ ]:
res = requests.get("https://lab.isaaclin.cn/nCoV/api/area?latest=0")
res = res.text
print(res)
res = json.loads(res)

In [ ]:
data_dict = []
for prov in res['results']:
    try:
        prov['cities']
    except Exception as e:
        # print(prov['provinceShortName'])
        continue
    for city in prov['cities']:
        data_dict.append({
            'date': prov['updateTime'],
            'province': prov['provinceShortName'],
            'city': city['cityName'],
            'confirm': city['confirmedCount'],
            'suspect': city['suspectedCount'],
            'cure': city['curedCount'],
            'dead': city['deadCount'],
        })
city_df = pd.DataFrame(data_dict)
city_df['date'] = city_df.date.astype('datetime64[ms]').dt.floor('d')
city_df = city_df.set_index('date')
city_df.to_csv("data/city_data_0204.csv")
with open("data/city_data_0204.json","w") as f:
    f.write(json.dumps(res['results']))
city_df.head()

In [ ]:
data_dict = []
for prov in res['results']:
    data_dict.append({
        'province': prov['provinceName'],
        'confirm': prov['confirmedCount'],
        'suspect': prov['suspectedCount'],
        'cure': prov['curedCount'],
        'dead': prov['deadCount'],
        'date': prov['updateTime']
    })
prov_data = pd.DataFrame(data_dict)

In [ ]:
prov_data['date'] = prov_data.date.astype('datetime64[ms]').dt.floor('d')
prov_data = prov_data.set_index('date')
prov_data.to_csv("data/prov_data_0204.csv")

In [ ]:
prov_data = pd.DataFrame.from_csv("data/prov_data_0204.csv")
prov_data.head(10)

In [ ]:
prov_data = prov_data.groupby([prov_data.index,prov_data.province]).agg({'confirm':max,'cure':max,'dead':max,'province':lambda x:x[0]})
prov_data

In [ ]:
prov_data = prov_data.set_index(prov_data.index.get_level_values(0))

In [ ]:
# look at time series for each province
fig, axes = plt.subplots(1,2,figsize=(14,8))
provinces = prov_data.province.unique()
for prov in provinces:
    sub_df = prov_data[prov_data.province==prov]
    # print('%s: %d' % (prov, len(sub_df)))
    # skip provinces without full week of data or starts with 0 
    if len(sub_df) < 7 or np.sum(sub_df.confirm == 0) > 0 :
        continue
    sub_df.plot(kind='line', y='confirm', style='-o', ax=axes[0], label=prov,logy=True)
axes[0].legend(bbox_to_anchor=(-0, 1),fontsize=14,frameon=False)
axes[0].set_ylabel("确诊人数",fontsize=16)
axes[0].set_xlabel("日期",fontsize=16)
# axes[0].set_ylim([1e0,5e3])
# look at hubei vs. outside hubei
prov_data[prov_data.province=='湖北省'].plot(ax=axes[1], kind='line', y='confirm', style='-^', label='湖北省',logy=True)
subdf = prov_data[prov_data.province!='湖北省']
subdf.groupby(subdf.index).confirm.sum().plot(ax=axes[1],kind='line', y='confirm', style='-D', label='湖北省以外',logy=True)
prov_data.groupby(prov_data.index).confirm.sum().plot(ax=axes[1],kind='line', y='confirm', style='-s', label='全国',logy=True)
axes[1].legend(fontsize=16,frameon=False)
axes[1].set_ylabel("确诊人数",fontsize=16)
axes[1].set_xlabel("日期",fontsize=16)
# axes[1].set_ylim([1e0,5e3])
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import r2_score
tb = 0

fig, axes = plt.subplots(1,2,figsize=(12,6))

# data fitting for data outside Hubei province
outside_hubei = subdf.groupby(subdf.index).confirm.sum()
x = np.arange(len(outside_hubei))
y = np.log10(outside_hubei)
axes[0].plot(x, 10**y, 'ks', label='湖北省外数据')
if tb != 0:
    x_ = np.arange(len(outside_hubei))[:-tb]
    y_ = np.log10(outside_hubei)[:-tb]
else:
    x_ = np.arange(len(outside_hubei))
    y_ = np.log10(outside_hubei)
coeff = np.polyfit(x_, y_, deg=1)
y_fit = coeff[0] * x + coeff[-1]
rsq = r2_score(y, y_fit)
axes[0].plot(x, 10**y_fit, 'k-', label='拟合(order=1): $r^2$=%.4f' % rsq)
coeff = np.polyfit(x_, y_, deg=2)
# coeff_saved = coeff.copy()
y_fit = coeff[0]*x**2 + coeff[1]*x + coeff[-1]
# y_fit = coeff[0]*x**3 + coeff[1]*x**2 + coeff[2]*x+coeff[-1]
rsq = r2_score(y, y_fit)
axes[0].plot(x, 10**y_fit, 'k--',label='拟合(order=2): $r^2$=%.4f' % rsq)
coeff = np.polyfit(x_, y_, deg=3)
coeff_saved = coeff.copy()
y_fit = coeff[0]*x**3 + coeff[1]*x**2 + coeff[2]*x+coeff[-1]
rsq = r2_score(y, y_fit)
axes[0].plot(x, 10**y_fit, 'b--',label='拟合(order=3): $r^2$=%.4f' % rsq)
axes[0].set_yscale('log')
axes[0].set_xlabel('日',fontsize=16)
axes[0].set_ylabel('确诊人数',fontsize=16)
axes[0].legend(fontsize=16)

# data fitting for Hubei province
inside_hubei = prov_data[prov_data.province=='湖北省'].confirm
y = np.log10(inside_hubei)
axes[1].plot(x, 10**y, 'ks', label='湖北省内数据')
coeff = np.polyfit(x, y, deg=1)
y_fit = coeff[0] * x + coeff[-1]
rsq = r2_score(y, y_fit)
axes[1].plot(x, 10**y_fit, 'k-', label='拟合(order=1): $r^2$=%.4f' % rsq)
coeff = np.polyfit(x, y, deg=2)
y_fit = coeff[0]*x**2 + coeff[1]*x + coeff[-1]
rsq = r2_score(y, y_fit)
axes[1].plot(x, 10**y_fit, 'k--',label='拟合(order=2): $r^2$=%.4f' % rsq)
axes[1].set_yscale('log')
axes[1].set_xlabel('日',fontsize=16)
axes[1].set_ylabel('确诊人数',fontsize=16)
axes[1].legend(fontsize=16)
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(8,6),sharex=True)
start_day = len(x)-1
x_pred = np.arange(start_day,start_day+2)
y_pred = coeff_saved[0]*x_pred**2 + coeff_saved[1]*x_pred + coeff_saved[-1]
y_fit = coeff_saved[0]*x**2 + coeff_saved[1]*x + coeff_saved[-1]
# y_fit = coeff_saved[0]*x**3 + coeff_saved[1]*x**2 + coeff_saved[2]*x + coeff_saved[-1]
axes[0].plot(outside_hubei.index[x], outside_hubei, 'ko', label='湖北省外数据')
axes[0].plot(outside_hubei.index[x], 10**y_fit, 'k--',label='拟合(order=3)')
date_pred = outside_hubei.index[:len(y_pred)]+pd.DateOffset(days=start_day)
err = outside_hubei - 10**y_fit 
# err = y_fit - np.log10(outside_hubei)
# use latest 5 days to predict std
_err = err.values[-5:]
axes[0].plot(date_pred, 10**y_pred, 'r--',label='预测')
axes[0].plot(date_pred[1], 10**y_pred[1], 'ro',label='%s: \n累计: %d$\pm$%d例\n增加: %d$\pm$%d例' % \
        (date_pred[1].date(), 10**y_pred[1], 2*int(_err.std(ddof=1)), 10**y_pred[1] - 10**y_pred[0], 2*int(_err.std(ddof=1))))
axes[0].set_yscale('log')
axes[0].set_ylabel('确诊人数/人',fontsize=16)
axes[0].legend(fontsize=16,frameon=False)
axes[1].plot(outside_hubei.index[x], err, 'ko', label='与拟合误差')
axes[1].axhline(0,color='k',ls=':')
axes[1].set_xlabel('日期',fontsize=16)
axes[1].set_ylabel('与拟合误差/人',fontsize=16)
plt.xticks(fontsize=12)
axes[1].legend(fontsize=16,frameon=False,loc="upper left")
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(8,6),sharex=True)
start_day = len(x)-1
x_pred = np.arange(start_day,start_day+2)
# y_pred = coeff_saved[0]*x_pred**2 + coeff_saved[1]*x_pred + coeff_saved[-1]
y_pred = coeff_saved[0]*x_pred**3 + coeff_saved[1]*x_pred**2 + coeff_saved[2]*x_pred + coeff_saved[-1]
# y_fit = coeff_saved[0]*x**2 + coeff_saved[1]*x + coeff_saved[-1]
y_fit = coeff_saved[0]*x**3 + coeff_saved[1]*x**2 + coeff_saved[2]*x + coeff_saved[-1]
axes[0].plot(outside_hubei.index[x], outside_hubei, 'ko', label='湖北省外数据')
axes[0].plot(outside_hubei.index[x], 10**y_fit, 'k--',label='拟合(order=3)')
date_pred = outside_hubei.index[:len(y_pred)]+pd.DateOffset(days=start_day)
err = outside_hubei - 10**y_fit 
# err = y_fit - np.log10(outside_hubei)
# use latest 5 days to predict std
_err = err.values[-5:]
axes[0].plot(date_pred, 10**y_pred, 'r--',label='预测')
axes[0].plot(date_pred[1], 10**y_pred[1], 'ro',label='%s: \n累计: %d$\pm$%d例\n增加: %d$\pm$%d例' % \
        (date_pred[1].date(), 10**y_pred[1], 2*int(_err.std(ddof=1)), 10**y_pred[1] - 10**y_pred[0], 2*int(_err.std(ddof=1))))
axes[0].set_yscale('log')
axes[0].set_ylabel('确诊人数/人',fontsize=16)
axes[0].legend(fontsize=16,frameon=False)
axes[1].plot(outside_hubei.index[x], err, 'ko', label='与拟合误差')
axes[1].axhline(0,color='k',ls=':')
axes[1].set_xlabel('日期',fontsize=16)
axes[1].set_ylabel('与拟合误差/人',fontsize=16)
plt.xticks(fontsize=12)
axes[1].legend(fontsize=16,frameon=False,loc="upper left")
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,8))
x = np.arange(len(outside_hubei)-1)
ax.plot(x, outside_hubei.values[1:] - outside_hubei.values[:-1],'ks',label='新增确诊人数(湖北省外)')
# plt.plot(x, 80*log(4*x+2))
ax.set_yscale('log')
ax.set_xlabel('天')
ax.set_ylabel("新增确诊人数")
ax.legend(frameon=True,loc='best')
ax2 = ax.twinx()
ax2.plot(x, (outside_hubei.values[1:] - outside_hubei.values[:-1])/outside_hubei.values[:-1]*100, 'k:',label='新增人数比例(湖北省外)')
ax2.legend(frameon=True,loc='center right')
ax2.set_ylabel("新增人数比例/%")

In [ ]:
# 各省增数histogram
hist = []
for prov in prov_data.province.unique():
    df = prov_data[prov_data.province==prov]
    if len(df)<2:
        continue
    if prov[-1] != '省' and prov[-1] != '市':
        continue
    rate = (df.confirm[-1] - df.confirm[-2])#/df.confirm[-2]*100
    hist.append({
        'province': prov,
        'rate': rate
    })
rate_df = pd.DataFrame(hist)
rate_df.head()

In [ ]:
plt.figure(figsize=(8,6))
rate_df = rate_df[np.isfinite(rate_df.rate)]
# rate_df.rate.hist(bins=np.linspace(5,35,31)+0.5)

In [ ]:
rate_df[rate_df.rate>20]